<a href="https://colab.research.google.com/github/geoskimoto/AWDB_SOAP_Request/blob/main/pull_awdb_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [131]:
import requests
import xml.dom.minidom as minidom
import json
import xml.etree.ElementTree as ET
# url = "https://www.wcc.nrcs.usda.gov/awdbWebService/services?WSDL"

import numpy as np
import pandas as pd

# https://www.dataquest.io/blog/python-api-tutorial/

In [2]:
#Example from https://www.youtube.com/watch?v=zZU0JZmIvKA
# payload = {'firstname':'John', 'lastname':'Smith'}
# r = requests.get('http://httpbin.org/get?', params=payload)
# print(r.url)
# print(r.content)
# print(r.text)

# response.text.split(' ')

In [1]:
# url = "https://www.wcc.nrcs.usda.gov/awdbWebService/services?WSDL"
# response = requests.get(url)
# if response.status_code != 200:
#     print('error {}'.format(response.status_code))
# else:
#     data = json.loads(response.text)
#     print(json.dumps(data, indent=4))

In [ ]:
#From the AWDB workbook:

# public Data[] getPeriodOfRecord(String p_stationTriplet, String p_beginDate,     
#                                 String p_endDate){                
#     Data[] values = m_webService.getData(Arrays.asList(p_stationTriplet),
#         "WTEQ", 1, null, Duration.DAILY, true, p_beginDate, p_endDate, true)
#         .toArray(new Data[0]);
#     return values;
# }

In [155]:
# Define Inputs
BeginDate = '01/01/2020'
EndDate = '03/01/2020'

# Create a dictionaries to store the data
# headers = {'Content-type': 'text/soap'}
# current_dictionary = {}
 
# Define Web Service URL
URL = "https://wcc.sc.egov.usda.gov/awdbWebService/services?WSDL"

# Define Parameters for SOAP Elements (getData:current and getCentralTendencyData:normals)
SOAP_current = '''
<?xml version="1.0" encoding="UTF-8"?>
<SOAP-ENV:Envelope xmlns:SOAP-ENV="http://schemas.xmlsoap.org/soap/envelope/" xmlns:q0="http://www.wcc.nrcs.usda.gov/ns/awdbWebService" xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">
  <SOAP-ENV:Body>
    <q0:getData>
      <stationTriplets>302:OR:SNTL</stationTriplets>
      <elementCd>WTEQ</elementCd>   
      <ordinal>1</ordinal>
      <duration>DAILY</duration>
      <getFlags>false</getFlags>
      <beginDate>01/01/2020</beginDate>
      <endDate>07/01/2020</endDate>
      <alwaysReturnDailyFeb29>true</alwaysReturnDailyFeb29>
    </q0:getData>
  </SOAP-ENV:Body>
</SOAP-ENV:Envelope>

'''.strip()

#other element codes: PREC, WTEQ (Water Equivalent/SWE)
 

# Post SOAP Elements to AWDB Web Service and process results - getData

SOAP_current = SOAP_current.replace("$BEGINDATE$", BeginDate)
SOAP_current = SOAP_current.replace("$ENDDATE$", EndDate)

response_current = requests.post(URL, data=SOAP_current, headers=headers)


In [156]:
response_current.text

'<soap:Envelope xmlns:soap="http://schemas.xmlsoap.org/soap/envelope/"><soap:Body><ns2:getDataResponse xmlns:ns2="http://www.wcc.nrcs.usda.gov/ns/awdbWebService"><return><beginDate>2020-01-01 00:00:00</beginDate><duration>DAILY</duration><endDate>2020-07-01 00:00:00</endDate><stationTriplet>302:OR:SNTL</stationTriplet><values>4.3</values><values>5.1</values><values>5.3</values><values>5.4</values><values>5.5</values><values>5.6</values><values>6.0</values><values>6.3</values><values>6.5</values><values>6.6</values><values>7.0</values><values>7.4</values><values>8.3</values><values>8.7</values><values>8.6</values><values>9.1</values><values>9.5</values><values>9.4</values><values>9.5</values><values>9.7</values><values>9.8</values><values>9.8</values><values>9.8</values><values>10.0</values><values>10.0</values><values>10.0</values><values>10.1</values><values>10.2</values><values>10.4</values><values>10.5</values><values>10.6</values><values>10.8</values><values>11.0</values><values>11

In [157]:
xml='<soap:Envelope xmlns:soap="http://schemas.xmlsoap.org/soap/envelope/"><soap:Body><ns2:getDataResponse xmlns:ns2="http://www.wcc.nrcs.usda.gov/ns/awdbWebService"><return><beginDate>2021-01-11 00:00:00</beginDate><duration>DAILY</duration><endDate>2021-01-11 00:00:00</endDate><stationTriplet>307:MT:SNTL</stationTriplet><values>19.1</values></return><return><beginDate>2021-01-11 00:00:00</beginDate><duration>DAILY</duration><endDate>2021-01-11 00:00:00</endDate><stationTriplet>309:WY:SNTL</stationTriplet><values>7.7</values></return></ns2:getDataResponse></soap:Body></soap:Envelope>'

In [158]:
# xmldoc_current = minidom.parseString(response_current.text)
# placemarks_current = xmldoc_current.getElementsByTagName('return')

# for returns_current in placemarks_current:
#     if returns_current.childNodes.length == 5:
#         if returns_current.getElementsByTagName('values')[0].firstChild:
#             date_output = returns_current.getElementsByTagName('beginDate')[0].firstChild.data
#             triplet_output = returns_current.getElementsByTagName('stationTriplet')[0].firstChild.data
#             id_output = triplet_output.split(':', 1)
#             id_output = id_output[0]
#             current_value = returns_current.getElementsByTagName('values')[0].firstChild.data
#             current_value = float(current_value)
#             current_dictionary.update({id_output: current_value})

#         else:
#             triplet_output = returns_current.getElementsByTagName('stationTriplet')[0].firstChild.data
#             id_output = triplet_output.split(':', 1)
#             id_output = id_output[0]
#             current_dictionary.update({id_output: 'na'})

#     else:
#         triplet_output = returns_current.getElementsByTagName('stationTriplet')[0].firstChild.data
#         id_output = triplet_output.split(':', 1)
#         id_output = id_output[0]
#         current_dictionary.update({id_output: 'na'})

# # print values to confirm its working
# print(current_dictionary)

In [159]:
xmldoc = minidom.parseString(response_current.text)
# placemarks_current = xmldoc_current.getElementsByTagName('return')

#### Create dictionary from request

In [160]:
{str(xmldoc.getElementsByTagName('stationTriplet')[0].firstChild.data):{1:xmldoc.getElementsByTagName('values')[0].firstChild.data}}

{'302:OR:SNTL': {1: '4.3'}}

In [161]:
val_length = len(xmldoc.getElementsByTagName('values'))
val_length

183

In [277]:
#Using OrderedDict
from collections import OrderedDict

date = datetime.datetime.strptime(BeginDate, "%d/%m/%Y").date() #.strftime("%Y,%m,%d")
station = xmldoc.getElementsByTagName('stationTriplet')
values = xmldoc.getElementsByTagName('values')
Date = []
for i in range(0, val_length): 
  date += datetime.timedelta(days=1)
  Date.append(str(date))

woohoo = {station[0].firstChild.data:OrderedDict({Date[j]:values[j].firstChild.data for j in range(0, val_length)})}
woohoo

{'302:OR:SNTL': OrderedDict([('2020-01-02', '4.3'),
              ('2020-01-03', '5.1'),
              ('2020-01-04', '5.3'),
              ('2020-01-05', '5.4'),
              ('2020-01-06', '5.5'),
              ('2020-01-07', '5.6'),
              ('2020-01-08', '6.0'),
              ('2020-01-09', '6.3'),
              ('2020-01-10', '6.5'),
              ('2020-01-11', '6.6'),
              ('2020-01-12', '7.0'),
              ('2020-01-13', '7.4'),
              ('2020-01-14', '8.3'),
              ('2020-01-15', '8.7'),
              ('2020-01-16', '8.6'),
              ('2020-01-17', '9.1'),
              ('2020-01-18', '9.5'),
              ('2020-01-19', '9.4'),
              ('2020-01-20', '9.5'),
              ('2020-01-21', '9.7'),
              ('2020-01-22', '9.8'),
              ('2020-01-23', '9.8'),
              ('2020-01-24', '9.8'),
              ('2020-01-25', '10.0'),
              ('2020-01-26', '10.0'),
              ('2020-01-27', '10.0'),
              ('2020

In [298]:
list(woohoo['302:OR:SNTL'])

['2020-01-02',
 '2020-01-03',
 '2020-01-04',
 '2020-01-05',
 '2020-01-06',
 '2020-01-07',
 '2020-01-08',
 '2020-01-09',
 '2020-01-10',
 '2020-01-11',
 '2020-01-12',
 '2020-01-13',
 '2020-01-14',
 '2020-01-15',
 '2020-01-16',
 '2020-01-17',
 '2020-01-18',
 '2020-01-19',
 '2020-01-20',
 '2020-01-21',
 '2020-01-22',
 '2020-01-23',
 '2020-01-24',
 '2020-01-25',
 '2020-01-26',
 '2020-01-27',
 '2020-01-28',
 '2020-01-29',
 '2020-01-30',
 '2020-01-31',
 '2020-02-01',
 '2020-02-02',
 '2020-02-03',
 '2020-02-04',
 '2020-02-05',
 '2020-02-06',
 '2020-02-07',
 '2020-02-08',
 '2020-02-09',
 '2020-02-10',
 '2020-02-11',
 '2020-02-12',
 '2020-02-13',
 '2020-02-14',
 '2020-02-15',
 '2020-02-16',
 '2020-02-17',
 '2020-02-18',
 '2020-02-19',
 '2020-02-20',
 '2020-02-21',
 '2020-02-22',
 '2020-02-23',
 '2020-02-24',
 '2020-02-25',
 '2020-02-26',
 '2020-02-27',
 '2020-02-28',
 '2020-02-29',
 '2020-03-01',
 '2020-03-02',
 '2020-03-03',
 '2020-03-04',
 '2020-03-05',
 '2020-03-06',
 '2020-03-07',
 '2020-03-

In [289]:
# #using zip

# date = datetime.datetime.strptime(BeginDate, "%d/%m/%Y").date() #.strftime("%Y,%m,%d")

# Date = []
# for i in range(0, val_length): 
#   date += datetime.timedelta(days=1)
#   Date.append(str(date))

# data2 = {station[0].firstChild.data: list(zip((Date[j],values[j].firstChild.data) for j in range(0, val_length)))}

# # list(zip((Date[j],values[j].firstChild.data) for j in range(0, val_length)))



In [181]:
date

datetime.datetime(2020, 1, 6, 0, 0)

In [190]:
import datetime

date = datetime.datetime.strptime(BeginDate, "%d/%m/%Y").date() #.strftime("%Y,%m,%d")

Date = []
for i in range(5): 
  date += datetime.timedelta(days=1)
  Date.append(str(date))

    # date += datetime.timedelta(days=1)
    # print(date) 

In [191]:
Date

['2020-01-02', '2020-01-03', '2020-01-04', '2020-01-05', '2020-01-06']

In [168]:
datetime.

datetime.datetime(2003, 8, 1, 12, 4, 5)

#### Create pd.DataFrame from request

In [245]:
val_length = len(xmldoc.getElementsByTagName('values'))

data = pd.DataFrame([xmldoc.getElementsByTagName('values')[i].firstChild.data for i in range(0,val_length)])

In [246]:
data

,0
0,4.3
1,5.1
2,5.3
3,5.4
4,5.5
...,...
178,1.6
179,0.9
180,3.0
181,5.8


In [247]:
date = datetime.datetime.strptime(BeginDate, "%d/%m/%Y").date() #.strftime("%Y,%m,%d")

Date = []
for i in range(val_length): 
  date += datetime.timedelta(days=1)
  Date.append(str(date))

# {str(xmldoc.getElementsByTagName('stationTriplet')[0].firstChild.data):{Date[j]:xmldoc.getElementsByTagName('values')[j].firstChild.data} for j in range(3)}

In [248]:
type(data.columns[0])

int

In [250]:
data['Date'] = Date
data.columns = ['SWE','Date']
# data.rename(columns={0:'SWE'}) #doesn't work for some reason, so renaming with just data.columns
data.set_index('Date', inplace=True)
data.tail(7)

,SWE
Date,
2020-06-26,2.2
2020-06-27,1.5
2020-06-28,1.6
2020-06-29,0.9
2020-06-30,3.0
2020-07-01,5.8
2020-07-02,5.2
